In [1]:
from __future__ import unicode_literals
from intermine.webservice import Service
from __future__ import print_function
import pandas as pd
service = Service("https://apps.araport.org:443/thalemine/service")
query = service.new_query("Gene")
query.add_view(
    "name", "computationalDescription", "briefDescription", "primaryIdentifier","curatorSummary"
)

In [44]:
keywords = ['Aquaporin','Prolin','dehydrin','late embryogenesis abundant','ABA']
genes = [[] for x in range(len(keywords))]

In [45]:
for row in query.rows():
    PI = row['primaryIdentifier']
    BD = str(row['briefDescription'])
    CD = str(row['computationalDescription'])
    if row['curatorSummary'] == None:
        CS = 'None'
    else: CS = row['curatorSummary'].encode('ascii', 'ignore')
    for i,keyword in enumerate(keywords):
        if keyword.upper() in BD.upper() or keyword.upper() in CS.upper() or keyword.upper() in CD.upper():
            genes[i].append([PI.encode('utf-8'),BD.encode('utf-8'),CS.encode('utf-8'),CD.encode('utf-8')])

In [46]:
service = Service("http://phytozome.jgi.doe.gov/phytomine/service")
def get_cds(name): # name should be transcript name
    template = service.get_template('Transcript_CDS_sequence')
    rows = template.rows(
        A = {"op": "=", "value": name}
        )
    for row in rows:
        return row["CDSs.sequence.residues"]

In [18]:
file_gff = './pascal/droughtnet/Athaliana_167_gene.gff3'

df_gff = pd.read_csv(file_gff,sep='\t',skiprows=1,header=None)
mask = (df_gff[2] == 'mRNA')
primary_transcript_list = [x.split(';')[1].replace('Name=','') for x in df_gff[mask][8] if x.split(';')[3] == 'longest=1']
genename_list = [x.split('.')[0] for x in primary_transcript_list]

dicGN2PT = dict(zip(genename_list,primary_transcript_list))

In [50]:
Outfile = open('keyword2info.txt','w')
def no_weird(words):
    a = words.replace('\u2018',' ').replace('\u2013',' ')
    return a
for i,keyword in enumerate(keywords):
    for gene in genes[i]:
        PI = gene[0]
        
        try:
            tgene = dicGN2PT[PI]
            print(keyword,';'.join(map(no_weird,gene)),tgene , get_cds(tgene),sep='\t',file=Outfile)
        except KeyError:
            pass
Outfile.close()

In [33]:
%cat ./pascal/droughtnet/Athaliana_167_gene.gff3 | grep 'AT1G418'

Chr1	phytozome8_0	gene	15582423	15585143	.	-	.	ID=AT1G41820;Name=AT1G41820
Chr1	phytozome8_0	mRNA	15582423	15585143	.	-	.	ID=PAC:19651278;Name=AT1G41820.1;pacid=19651278;longest=1;Parent=AT1G41820
Chr1	phytozome8_0	gene	15603617	15607966	.	-	.	ID=AT1G41830;Name=AT1G41830
Chr1	phytozome8_0	mRNA	15603617	15607966	.	-	.	ID=PAC:19652521;Name=AT1G41830.1;pacid=19652521;longest=1;Parent=AT1G41830
Chr1	phytozome8_0	gene	15650172	15650405	.	+	.	ID=AT1G41875;Name=AT1G41875
Chr1	phytozome8_0	mRNA	15650172	15650405	.	+	.	ID=PAC:19653868;Name=AT1G41875.1;pacid=19653868;longest=1;Parent=AT1G41875
Chr1	phytozome8_0	gene	15651027	15652542	.	-	.	ID=AT1G41880;Name=AT1G41880
Chr1	phytozome8_0	mRNA	15651027	15652542	.	-	.	ID=PAC:19654231;Name=AT1G41880.1;pacid=19654231;longest=1;Parent=AT1G41880


In [49]:
get_cds('AT3G19580.1')

u'ATGGCCCTCGAAGCGATGAACACTCCAACTTCTTCTTTCACCAGAATCGAAACGAAAGAAGATTTGATGAACGACGCCGTTTTCATTGAGCCGTGGCTTAAACGCAAACGCTCCAAACGTCAGCGTTCTCACAGCCCTTCTTCGTCTTCTTCCTCACCGCCTCGATCTCGACCCAAATCCCAGAATCAAGATCTTACGGAAGAAGAGTATCTCGCTCTTTGTCTCCTCATGCTCGCTAAAGATCAACCGTCGCAAACGCGATTTCATCAACAGTCGCAATCGTTAACGCCGCCGCCAGAATCAAAGAACCTTCCGTACAAGTGTAACGTCTGTGAAAAAGCGTTTCCTTCCTATCAGGCTTTAGGCGGTCACAAAGCAAGTCACCGAATCAAACCACCAACCGTAATCTCAACAACCGCCGATGATTCAACAGCTCCGACCATCTCCATCGTCGCCGGAGAAAAACATCCGATTGCTGCCTCCGGAAAGATCCACGAGTGTTCAATCTGTCATAAAGTGTTTCCGACGGGTCAAGCTTTAGGCGGTCACAAACGTTGTCACTACGAAGGCAACCTCGGCGGCGGAGGAGGAGGAGGAAGCAAATCAATCAGTCACAGTGGAAGCGTGTCGAGCACGGTATCGGAAGAAAGGAGCCACCGTGGATTCATCGATCTAAACCTACCGGCGTTACCTGAACTCAGCCTTCATCACAATCCAATCGTCGACGAAGAGATCTTGAGTCCGTTGACCGGTAAAAAACCGCTTTTGTTGACCGATCACGACCAAGTCATCAAGAAAGAAGATTTATCTTTAAAAATCTAA'